<a href="https://colab.research.google.com/github/mkbane/mmu-big-data/blob/main/wordcount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls

counts_via_RDD.txt  spark-3.2.1-bin-hadoop3.2	   t8.shakespeare.txt
sample_data	    spark-3.2.1-bin-hadoop3.2.tgz


In [ ]:
# set-up spark (NB if Apache amend versions on download site we will need to amend path in wget command)
!clear
!echo welcome
!ls
!rm -f spark-3.2.[01]-bin-hadoop3.2.tgz* 
!rm -rf spark-3.2.[01]-bin-hadoop3.2
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar -xf spark-3.2.1-bin-hadoop3.2.tgz
!ls -alt

welcome
sample_data
--2022-02-21 19:12:54--  https://downloads.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 300971569 (287M) [application/x-gzip]
Saving to: ‘spark-3.2.1-bin-hadoop3.2.tgz’

spark-3.2.1-bin-had 100%[===================>] 287.03M  25.7MB/s    in 12s     

2022-02-21 19:13:07 (23.9 MB/s) - ‘spark-3.2.1-bin-hadoop3.2.tgz’ saved [300971569/300971569]

total 293940
drwxr-xr-x  1 root root      4096 Feb 21 19:13 .
drwxr-xr-x  1 root root      4096 Feb 21 19:11 ..
drwxr-xr-x  1 root root      4096 Feb  1 14:32 sample_data
drwxr-xr-x  4 root root      4096 Feb  1 14:31 .config
-rw-r--r--  1 root root 300971569 Jan 20 21:37 spark-3.2.1-bin-hadoop3.2.tgz
drwxr-xr-x 13  501 1000      4096 Jan 20 20:10 spark-3.2.1-

In [ ]:
!hdfs dfs -ls

/bin/bash: hdfs: command not found


In [ ]:
#do a map-reduce word count

# wget file
!wget https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt
!ls

--2022-02-21 19:13:24--  https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt
Resolving ocw.mit.edu (ocw.mit.edu)... 151.101.2.133, 151.101.66.133, 151.101.130.133, ...
Connecting to ocw.mit.edu (ocw.mit.edu)|151.101.2.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5458199 (5.2M) [text/plain]
Saving to: ‘t8.shakespeare.txt’

t8.shakespeare.txt  100%[===================>]   5.21M  --.-KB/s    in 0.08s   

2022-02-21 19:13:25 (68.5 MB/s) - ‘t8.shakespeare.txt’ saved [5458199/5458199]

sample_data		   spark-3.2.1-bin-hadoop3.2.tgz
spark-3.2.1-bin-hadoop3.2  t8.shakespeare.txt


In [ ]:
# init spark (ensure SPARK_HOME set to same version as we download earlier)
!pip3 install findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
# the next line gives us 'local' mode. try 'local[2]' to use 2 cores or 'master:NNNN' to run on Spark standalone cluster at port NNNN
spark_conf = SparkConf().setMaster('local[4]').setAppName('MyApp')
sc = SparkContext(conf=spark_conf)
# see what we have by examining the Spark User Interface
from pyspark.sql import *
from pyspark.sql.functions import *
SparkSession.builder.getOrCreate()

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

RDD EXAMPLE

In [ ]:
text_file = sc.textFile("t8.shakespeare.txt")
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)
counts.saveAsTextFile("counts_via_RDD_v2.txt")  # will not overwrite

  

DF EXAMPLE

In [ ]:
spark = SparkSession.builder.appName("words").getOrCreate()


In [ ]:
  input = spark.read.text("t8.shakespeare.txt")

In [ ]:
input.show()

+--------------------+
|               value|
+--------------------+
|This is the 100th...|
|is presented in c...|
|Library of the Fu...|
|often releases Et...|
|                    |
|         Shakespeare|
|                    |
|*This Etext has c...|
|                    |
|<<THIS ELECTRONIC...|
|SHAKESPEARE IS CO...|
|PROVIDED BY PROJE...|
|WITH PERMISSION. ...|
|DISTRIBUTED SO LO...|
|PERSONAL USE ONLY...|
|COMMERCIALLY.  PR...|
|SERVICE THAT CHAR...|
|                    |
|*Project Gutenber...|
|in the presentati...|
+--------------------+
only showing top 20 rows



In [ ]:
words = input.select(explode(split(input.value, " ")).alias("word"))

In [ ]:
words.show()

+-----------+
|       word|
+-----------+
|       This|
|         is|
|        the|
|      100th|
|      Etext|
|       file|
|  presented|
|         by|
|    Project|
| Gutenberg,|
|        and|
|         is|
|  presented|
|         in|
|cooperation|
|       with|
|      World|
|   Library,|
|      Inc.,|
|       from|
+-----------+
only showing top 20 rows



In [ ]:
wordCount = words.groupBy("word").count()

In [ ]:
wordCount.show()

+-----------+-----+
|       word|count|
+-----------+-----+
|PERMISSION.|  221|
|       AWAY|    1|
|         By|  809|
|      those|  458|
|        art|  726|
|       some| 1071|
|      still|  331|
|   commits.|    1|
|     pluck,|    1|
|     fight,|   42|
|       hope|  237|
|     travel|   20|
|      cures|    6|
|     ransom|   34|
|     youth,|   73|
|    uprear,|    1|
|      spoil|   13|
|    tresses|    2|
|        few|   44|
|      cold,|   24|
+-----------+-----+
only showing top 20 rows



In [ ]:
words.groupBy("word").count().sort("count", ascending=False).show()

+----+------+
|word| count|
+----+------+
|    |517065|
| the| 23242|
|   I| 19540|
| and| 18297|
|  to| 15623|
|  of| 15544|
|   a| 12532|
|  my| 10824|
|  in|  9576|
| you|  9081|
|  is|  7851|
|that|  7531|
| And|  7068|
| not|  6948|
|with|  6722|
| his|  6218|
|your|  6009|
|  be|  6002|
| for|  5616|
|have|  5236|
+----+------+
only showing top 20 rows



In [ ]:
words.groupBy("word").count().orderBy("count", ascending=False).show()

+----+------+
|word| count|
+----+------+
|    |517065|
| the| 23242|
|   I| 19540|
| and| 18297|
|  to| 15623|
|  of| 15544|
|   a| 12532|
|  my| 10824|
|  in|  9576|
| you|  9081|
|  is|  7851|
|that|  7531|
| And|  7068|
| not|  6948|
|with|  6722|
| his|  6218|
|your|  6009|
|  be|  6002|
| for|  5616|
|have|  5236|
+----+------+
only showing top 20 rows

